In [ ]:
!git clone https://github.com/SZTankWang/ml-weibo-project-.git
!pip install paddlepaddle-tiny==1.6.1
!pip install jieba --upgrade

fatal: destination path 'ml-weibo-project-' already exists and is not an empty directory.
Requirement already up-to-date: jieba in /usr/local/lib/python3.7/dist-packages (0.42.1)


In [ ]:
import jieba
import paddle
import re
import pandas as pd
import time
import sklearn
from sklearn.linear_model import LinearRegression
import math as m
import random as r
import numpy as np

# paddle.enable_static()
jieba.enable_paddle()# 启动paddle模式。 0.40版之后开始支持，早期版本不支持

Paddle enabled successfully......


In [ ]:
df = pd.read_excel("/content/ml-weibo-project-/data/raw_data.xlsx",skiprows=1,usecols=["微博正文","点赞数","转发数","评论数"])
df.head()

,微博正文,点赞数,转发数,评论数
0,【#你好，明天#】岁末年初，即便你的年终盘点一言难尽，但新年的flag仍然计日可期。需要抓住...,5389,1997,406
1,【今天，发条微博，向这些“闪亮的名字”致敬】南仁东、林俊德、张超、王继才、黄群、宋月才、姜开...,6826,1268,289
2,【夜读：这一年，谢谢自己】转眼间，2018年就要过去了。有时候我们很怕，怕自己的成长速度跟不...,6565,4326,580
3,【小调查：跨年夜，你如何度过的？】#2018最后一天#，你是如何度过的？陪在家人身边？宅在家...,919,2562,5281
4,【揭秘！习主席书架上的新变化】今晚，习近平主席发表了#2019新年贺词#。这一次，习主席书架...,6513,1758,440


In [ ]:
df = df.rename(columns = {'微博正文':'text', '点赞数':'like', '转发数':'comment','评论数':'forward'}, inplace=False)
df.head()

,text,like,comment,forward
0,【#你好，明天#】岁末年初，即便你的年终盘点一言难尽，但新年的flag仍然计日可期。需要抓住...,5389,1997,406
1,【今天，发条微博，向这些“闪亮的名字”致敬】南仁东、林俊德、张超、王继才、黄群、宋月才、姜开...,6826,1268,289
2,【夜读：这一年，谢谢自己】转眼间，2018年就要过去了。有时候我们很怕，怕自己的成长速度跟不...,6565,4326,580
3,【小调查：跨年夜，你如何度过的？】#2018最后一天#，你是如何度过的？陪在家人身边？宅在家...,919,2562,5281
4,【揭秘！习主席书架上的新变化】今晚，习近平主席发表了#2019新年贺词#。这一次，习主席书架...,6513,1758,440


In [ ]:
stopwords = []
with open('/content/ml-weibo-project-/stopwords.txt') as file:
    for i in file:
        stopwords.append(i.strip('\n'))

cutwordslist = []
oplis = []
sentences = df.text.to_list()
f = open("/content/ml-weibo-project-/tfidf.txt", "w")

for line in sentences[]:
    linelis = list(line)
    newlis = []
    for i in linelis:
        if re.match('[\w]', i):
            newlis.append(i)
        else:
            newlis.append('')
    newline = ''.join(newlis)
    
    # cutwordslist.append([word for word in jieba.cut(newline, use_paddle=True) if word not in stopwords])

    line = ' '.join([word for word in jieba.cut(newline, use_paddle=True) if word not in stopwords])
    f.write(line)
    f.write('\n')
    # oplis.append(line)
    # afterTfIdf.append()
# print(oplis)
# f = open("/content/ml-weibo-project-/tfidf.txt", "w")
# f.write(line)
f.close()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
f = open("/content/ml-weibo-project-/tfidf.txt", 'r')
allsentences = f.readlines()

for i in range(len(allsentences)):
  allsentences[i] = allsentences[i].strip('\n')
y = df[['like','comment','forward']][:106375]

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(allsentences, y, test_size=.7, train_size=.3, random_state=0, shuffle=True, stratify=None)
f.close()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

 
#该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
vectorizer = CountVectorizer()
#该类会统计每个词语的tf-idf权值
tf_idf_transformer = TfidfTransformer()
#将文本转为词频矩阵并计算tf-idf
tf_idf = tf_idf_transformer.fit_transform(vectorizer.fit_transform(X_train))
#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重
X_train_weight = tf_idf.toarray()
 
#对测试集进行tf-idf权重计算
tf_idf = tf_idf_transformer.transform(vectorizer.transform(X_test))
X_test_weight = tf_idf.toarray()  # 测试集TF-IDF权重矩阵
 
#print('输出x_train文本向量：')
#print(X_train_weight)
#print('输出x_test文本向量：')
#print(X_test_weight)


model = LinearRegression().fit(X_train_weight, y_train)
y_train_pred = model.predict(X_train_weight)
y_test_pred = model.predict(X_test_weight)

train_error = sklearn.metrics.mean_squared_error(y_train, y_train_pred)
test_error = sklearn.metrics.mean_squared_error(y_test, y_test_pred)


4.632230694701547e-19
7555792094.750149


# New Section